In [51]:
#Imports

%matplotlib inline
import math 

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import Perceptron


import category_encoders as ce


# Set de Datos

In [52]:
# Carga de datos.

train = pd.read_csv('./train.csv'
            ,dtype={
             "id":np.int32
            ,"titulo":str
            ,"fecha":str 
            ,"ciudad": str
            ,"provincia": str
            ,"descripcion": str  
            ,"direccion": str  
            ,"tipodepropiedad": 'category' })

test = pd.read_csv('./test.csv'
            ,dtype={
             "id":np.int32
            ,"titulo":str
            ,"fecha":str 
            ,"ciudad": str
            ,"provincia": str
            ,"descripcion": str  
            ,"direccion": str 
            ,"tipodepropiedad": 'category' })


# Metrica RMSLE

In [ ]:
# Métrica de evaluación
def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

# Funciones

In [ ]:
def fill_nulls( a ):

    a[['metroscubiertos']] = a[['metroscubiertos']].fillna(value= (a['metroscubiertos'].mean())  )
    a[['antiguedad']] = a[['antiguedad']].fillna(value= (a['antiguedad'].mean()) )
    a[['habitaciones']] = a[['habitaciones']].fillna(value= (a['habitaciones'].mean()) )
    a[['banos']] = a[['banos']].fillna(value= (a['habitaciones'].max()))
    a[['idzona']] = a[['idzona']].fillna(value=(a['idzona'].median()))
    a[['garages']] = a[['garages']].fillna(value=(a['garages'].min()))
    a[['metrostotales']] = a[['metrostotales']].fillna(value=(a['metrostotales'].mean()))
    a[['lng']] = a[['lng']].fillna(value=(a['lng'].median()))
    a[['lat']] = a[['lat']].fillna(value=(a['lat'].median()))
    a[['centroscomercialescercanos']] = a[['centroscomercialescercanos']].fillna(value=(a['metrostotales'].median()) or 0 )
    a[['piscina']] = a[['piscina']].fillna(value= (a['piscina'].min()) )
    a[['gimnasio']] = a[['gimnasio']].fillna(value= (a['gimnasio'].min()) )
    a[['usosmultiples']] = a[['usosmultiples']].fillna(value= (a['usosmultiples'].min()) )
    
    a[['provincia']] = a[['provincia']].fillna(value= str(a['provincia'].mode()) )
    a[['ciudad']] = a[['ciudad']].fillna(value= str(a['ciudad'].mode()) )
    a[['descripcion']] = a[['descripcion']].fillna(value= str(a['descripcion'].mode()) )
    a[['titulo']] = a[['titulo']].fillna(value= str(a['titulo'].mode()) )
    a[['direccion']] = a[['direccion']].fillna(value= str(a['direccion'].mode()) )
    a[['fecha']] = a[['fecha']].fillna(value= str( a['fecha'].mode() ) )
    
    return a

In [ ]:
def calcularError(model,X_test,X_train ,y_test,y_train):
    #Calculo de Errores contra mi set de test
    pred = model.predict(X_test)

    #Calculo de Errores contra mi set de entrenamiento

    rmsle_test = RMSLE(y_test, pred)
    rmsle_train = RMSLE(y_train, model.predict(X_train) )

    print(f"RMSLE Error (train): {rmsle_train:.5f}")
    print(f"RMSLE Error (Test): {rmsle_test:.5f}")


    mae_test = mean_absolute_error(y_test, pred)
    mae_train = mean_absolute_error(y_train, model.predict(X_train) )

    print(f"MAE Error (train): {mae_train:.5f}")
    print(f"MAE Error (Test): {mae_test:.5f}")

In [ ]:
def generarCsv(model):
    
    c = test

    c[['tipodepropiedad']] = c[['tipodepropiedad']].fillna(value= 'Casa')
    c[['fecha']] = c[['fecha']].fillna(value= ( a['fecha'].mode() ) )

    c = fill_nulls(c)

    one_hot_enc = ce.OneHotEncoder()
    one_hot_encoded = one_hot_enc.fit_transform(c['tipodepropiedad'])
    c = c.join(one_hot_encoded.add_suffix("_oh"))

    c['fecha'] = pd.to_datetime(c['fecha'])
    c['anio'] = c['fecha'].dt.year
    c['mes'] = c['fecha'].dt.month
    c['dia'] = c['fecha'].dt.dayofweek
    
    c['titulo'] = c['titulo'].apply(len)
    c['descripcion'] = c['descripcion'].apply(len)
    c['direccion'] = c['direccion'].apply(len)
    c['ciudad'] = c['ciudad'].apply(len)
    c['provincia'] = c['provincia'].apply(len)
    
    c['metrostotales'] = np.log(c['metrostotales'])
    c['metroscubiertos'] = np.log(c['metroscubiertos'])


    b = c[features]

    #Exporto csv con la prediccion
    pred = model.predict(b) 
    #Creo un dataframe con el formato ( id , precio (El precio predecido ))
    res = test['id'].to_frame()
    res.insert(1,'target', pred , True) 
    #Exportamos la prediccion en formato csv sin labels
    res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)
    print('CSV creado Exitosamente.')

# Constantes

In [ ]:
 features = [ 'anio',
    
              'mes',
              'dia',
              'titulo',
              'descripcion',
              'direccion',
              'ciudad',
              'provincia',
              
              'metrostotales',
              'escuelascercanas',
              'antiguedad',
              'centroscomercialescercanos',
              'garages',
              'piscina',
              'metroscubiertos',
              'banos',
              'habitaciones',
              'lat','lng','idzona',
              'gimnasio',
              'usosmultiples',
              'tipodepropiedad_1_oh',
              'tipodepropiedad_2_oh',
              'tipodepropiedad_3_oh',
              'tipodepropiedad_4_oh',
              'tipodepropiedad_5_oh',
              'tipodepropiedad_6_oh',
              'tipodepropiedad_7_oh',
              'tipodepropiedad_8_oh',
              'tipodepropiedad_9_oh',
              'tipodepropiedad_10_oh',
              'tipodepropiedad_11_oh',
              'tipodepropiedad_12_oh',
              'tipodepropiedad_13_oh',
              'tipodepropiedad_14_oh',
              'tipodepropiedad_15_oh',
              'tipodepropiedad_16_oh',
              'tipodepropiedad_17_oh',
              'tipodepropiedad_18_oh',
              'tipodepropiedad_19_oh',
              'tipodepropiedad_20_oh',
              'tipodepropiedad_21_oh',
              'tipodepropiedad_22_oh' ]
    
len( features )

# Preprocesamiento de Datos

In [ ]:
a = train

a = a.loc[ ( a['tipodepropiedad'] != 'Hospedaje'  ) & ( a['tipodepropiedad'] != 'Garage'  )  ]

a[['tipodepropiedad']] = a[['tipodepropiedad']].fillna(value='Casa')

nulls = pd.DataFrame( a.isna().sum().sort_values(ascending=False) , columns=['Nulls'] )
nulls['%porcentaje'] = round(100*nulls['Nulls'] / len(a), 2)
nulls

### Manejo de Nulls

In [ ]:
a = fill_nulls(a)

print('Nulls cambiados !')

In [ ]:
nulls = pd.DataFrame( a.isna().sum().sort_values(ascending=False) , columns=['Nulls'] )
nulls['%porcentaje'] = round(100*nulls['Nulls'] / len(a), 2)
nulls.head()

### Encoding variables categoricas

##### One Hot Encoding

In [ ]:
one_hot_enc = ce.OneHotEncoder()
one_hot_encoded = one_hot_enc.fit_transform(a['tipodepropiedad'])
a= a.join(one_hot_encoded.add_suffix("_oh"))
print('One Hot encoding done!')

### Transformacion de features

In [ ]:
a['fecha'] = pd.to_datetime(a['fecha'])

a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month
a['dia'] = a['fecha'].dt.dayofweek

a['titulo'] = a['titulo'].apply(len)
a['descripcion'] = a['descripcion'].apply(len)
a['direccion'] = a['direccion'].apply(len)
a['ciudad'] = a['ciudad'].apply(len)
a['provincia'] = a['provincia'].apply(len)

a['metrostotales'] = np.log(a['metrostotales'])
a['metroscubiertos'] = np.log(a['metroscubiertos'])

##### Vistazo de como queda

In [ ]:
print( f"\nTamaño : {a.shape}" )

# 1- Prediccion sin dividir el set

### Split del set de datos General ( no sub dividi por años )

In [ ]:

X = a[features]

y = a['precio']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=1)

### Random Forest

In [ ]:
regr = RandomForestRegressor( random_state=0,bootstrap =False,min_samples_leaf=2 ,n_jobs=-1,
                             max_depth = 50,max_features = 21 ,min_samples_split = 2 ,
                             n_estimators = 175 )

#Entreno

model = regr.fit(X_train, y_train)

print('Finalizo el entrenamiento\n\n')

calcularError(model,X_test,X_train ,y_test,y_train)


### Random forest  + Random Grid + Cross Validation

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 400, num = 15)]
max_features = [int(x) for x in np.linspace(start = 4, stop = 44, num = 15)]
max_depth = [int(x) for x in np.linspace(50, 100, num = 10)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [ False]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, random_state=42, n_jobs = -1)


# Fit the random search model
model = rf_random.fit(X_train, y_train)


# Muestra los mejores paramentros encontrados
print( rf_random.best_params_ )
print('Finalizo el random search + Cross validation \n\n')
calcularError(model,X_test,X_train ,y_test,y_train)

In [ ]:
generarCsv(model)

# VottingRegresor

In [ ]:
# Create Modelos

r1 = RandomForestRegressor(random_state=0,bootstrap =False,min_samples_leaf=2 ,n_jobs=-1,
                           max_depth = 50,max_features = 21 ,min_samples_split = 2 ,
                           n_estimators = 175 )

r2 = KNeighborsRegressor(50)

r3 = DummyRegressor(strategy='mean')

r4 = linear_model.LinearRegression()


er = VotingRegressor([('rf', r1),('rf2', r3),('rf3', r4),('rf4', r2)],weights =[1,0.01,0.1,0.75])
     
    
# Train model
model = er.fit(X_train, y_train)

print('Termino de entrenar el modelo\n\n')
calcularError(model,X_test,X_train ,y_test,y_train)

### Generacion de csv

In [ ]:
generarCsv(model)


# 2 - Predicciones por año

In [ ]:
#Copio el train para trabajar con él
a = train

a = a.loc[ ( a['tipodepropiedad'] != 'Hospedaje'  ) & ( a['tipodepropiedad'] != 'Garage'  )  ]
a[['tipodepropiedad']] = a[['tipodepropiedad']].fillna(value='Casa')

a = fill_nulls(a)

print('filtrado y nulls hecho!\n')

nulls = pd.DataFrame( a.isna().sum().sort_values(ascending=False) , columns=['Nulls'] )
nulls['%porcentaje'] = round(100*nulls['Nulls'] / len(a), 2)
nulls.head(3)

In [ ]:
# One hot encoding

one_hot_enc = ce.OneHotEncoder()
one_hot_encoded = one_hot_enc.fit_transform(a['tipodepropiedad'])
a= a.join(one_hot_encoded.add_suffix("_oh"))
print('One Hot encoding done!')

In [ ]:
#Transformacion de variables

a['fecha'] = pd.to_datetime(a['fecha'])

a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month
a['dia'] = a['fecha'].dt.dayofweek


a['titulo'] = a['titulo'].apply(len)
a['descripcion'] = a['descripcion'].apply(len)
a['direccion'] = a['direccion'].apply(len)
a['ciudad'] = a['ciudad'].apply(len)
a['provincia'] = a['provincia'].apply(len)

a['metrostotales'] = np.log(a['metrostotales'])
a['metroscubiertos'] = np.log(a['metroscubiertos'])

print('Varibales transformadas !')

In [ ]:
# Dividi por año

a2012 = a.loc[ a['anio'] == 2012 ]
a2013 = a.loc[ a['anio'] == 2013 ]
a2014 = a.loc[ a['anio'] == 2014 ]
a2015 = a.loc[ a['anio'] == 2015 ]
a2016 = a.loc[ a['anio'] == 2016 ]

print('Particionado !')

# Para el año 2012

In [ ]:
# Para el set de datos 2012 lleno los nulls

a2012 = fill_nulls(a2012)

# Creo los set de test y train

X = a2012[features]

y = a2012['precio']

# Pariciono los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

print('Separacion del set de datos exitosamente.')

In [ ]:
# Random Search + Cross Validation

# {'n_estimators': 244, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 21, 'max_depth': 88, 'bootstrap': False}

n_estimators = [int(x) for x in np.linspace(start = 50, stop = 400, num = 10)]
max_features = [int(x) for x in np.linspace(start = 4, stop = 36, num = 10)]
max_depth = [int(x) for x in np.linspace(50, 100, num = 10)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

#Modelo random forest

rf = RandomForestRegressor()

#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 2, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, random_state=42, n_jobs = -1)


# Fit the random search model

model2012 = rf_random.fit(X_train, y_train)

print(rf_random.best_params_)
print('Modelo hecho!')

In [ ]:
#Resultados
calcularError(model2012,X_test,X_train ,y_test,y_train)

# Para 2013

In [ ]:
# Para el set de datos 2013 lleno los nulls

a2013 = fill_nulls(a2013)

# Creo los set de test y train

X = a2013[features]

y = a2013['precio']

# Pariciono los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

print('Separacion del set de datos exitosamente.')

In [ ]:
# Random Search + Cross Validation

n_estimators = [int(x) for x in np.linspace(start = 50, stop = 400, num = 10)]
max_features = [int(x) for x in np.linspace(start = 4, stop = 36, num = 10)]
max_depth = [int(x) for x in np.linspace(50, 100, num = 10)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


rf = RandomForestRegressor()
#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 2, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, random_state=42, n_jobs = -1)


# Fit the random search model

model2013 = rf_random.fit(X_train, y_train)

print(rf_random.best_params_)
print('Modelo hecho!')

#{'n_estimators': 244, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 21, 'max_depth': 88, 'bootstrap': False}

In [ ]:
#Resultados

calcularError(model2013,X_test,X_train ,y_test,y_train)

# Para 2014

In [ ]:
# Para el set de datos 2012 lleno los nulls

a2014 = fill_nulls(a2014)

# Creo los set de test y train

X = a2014[features]

y = a2014['precio']

# Pariciono los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

print('Separacion del set de datos exitosamente.')

In [ ]:
# Random Search + Cross Validation

n_estimators = [int(x) for x in np.linspace(start = 50, stop = 400, num = 20)]
max_features = [int(x) for x in np.linspace(start = 4, stop = 36, num = 15)]
max_depth = [int(x) for x in np.linspace(50, 100, num = 20)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


rf = RandomForestRegressor()
#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 2, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, random_state=42, n_jobs = -1)


# Fit the random search model

model2014 = rf_random.fit(X_train, y_train)

print(rf_random.best_params_)
print('Modelo hecho!')

#{'n_estimators': 381, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 17, 'max_depth': 81, 'bootstrap': False}

In [ ]:
#Resultados

calcularError(model2014,X_test,X_train ,y_test,y_train)

# Para 2015

In [ ]:
# Para el set de datos 2012 lleno los nulls

a2015 = fill_nulls(a2015)

# Creo los set de test y train

X = a2015[features]

y = a2015['precio']

# Pariciono los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

print('Separacion del set de datos exitosamente.')

In [ ]:
# Random Search + Cross Validation

n_estimators = [int(x) for x in np.linspace(start = 50, stop = 400, num = 20)]
max_features = [int(x) for x in np.linspace(start = 4, stop = 36, num = 15)]
max_depth = [int(x) for x in np.linspace(50, 100, num = 20)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


rf = RandomForestRegressor()
#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 2, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, random_state=42, n_jobs = -1)


# Fit the random search model

model2015 = rf_random.fit(X_train, y_train)

print(rf_random.best_params_)
print('Modelo hecho!')

#{'n_estimators': 381, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 17, 'max_depth': 81, 'bootstrap': False}

In [ ]:
#Resultados

calcularError(model2015,X_test,X_train ,y_test,y_train)

# Para 2016

In [ ]:
# Para el set de datos 2012 lleno los nulls
a2016 = pd.concat([a2012,a2013,a2014,a2015,a2016], ignore_index=False)
a2016 = a2016.sort_index()
a2016 = fill_nulls(a2016)

# Creo los set de test y train

X = a2016[features]

y = a2016['precio']

# Pariciono los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

print('Separacion del set de datos exitosamente.')

In [ ]:
# Random Search + Cross Validation

n_estimators = [int(x) for x in np.linspace(start = 50, stop = 400, num = 10)]
max_features = [int(x) for x in np.linspace(start = 4, stop = 44, num = 10)]
max_depth = [int(x) for x in np.linspace(50, 100, num = 20)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [ False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


rf = RandomForestRegressor()
#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 2, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, random_state=42, n_jobs = -1)


# Fit the random search model

model2016 = rf_random.fit(X_train, y_train)

print(rf_random.best_params_)
print('Modelo hecho!')

#{'n_estimators': 381, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 16, 'max_depth': 60, 'bootstrap': False}

In [ ]:
#Resultados
calcularError(model2016,X_test,X_train ,y_test,y_train)

# Predicciones del Test

In [ ]:
#Separo el set de test

c = test

c = c.loc[ ( c['tipodepropiedad'] != 'Hospedaje'  ) & ( c['tipodepropiedad'] != 'Garage'  )  ]
c[['tipodepropiedad']] = c[['tipodepropiedad']].fillna(value='Casa')

c = fill_nulls(c)

# One hot encoding

one_hot_enc = ce.OneHotEncoder()
one_hot_encoded = one_hot_enc.fit_transform(c['tipodepropiedad'])
c= c.join(one_hot_encoded.add_suffix("_oh"))
print('One Hot encoding done!')

#Transformacion de variables

c['fecha'] = pd.to_datetime(c['fecha'])

c['anio'] = c['fecha'].dt.year
c['mes'] = c['fecha'].dt.month
c['dia'] = c['fecha'].dt.dayofweek

c['titulo'] = c['titulo'].apply(len)
c['descripcion'] = c['descripcion'].apply(len)
c['direccion'] = c['direccion'].apply(len)
c['ciudad'] = c['ciudad'].apply(len)
c['provincia'] = c['provincia'].apply(len)

c['metrostotales'] = np.log(c['metrostotales'])
c['metroscubiertos'] = np.log(c['metroscubiertos'])


print('Varibales transformadas !')

# Dividi por año

c2012 = c.loc[ c['anio'] == 2012 ]
c2013 = c.loc[ c['anio'] == 2013 ]
c2014 = c.loc[ c['anio'] == 2014 ]
c2015 = c.loc[ c['anio'] == 2015 ]
c2016 = c.loc[ c['anio'] == 2016 ]

print('Particionado !')

# Predicciones por año

In [ ]:
# Para el año 2016 

c2016 = fill_nulls(c2016)

# Creo los set de test y train

d = c2016[features]

#Exporto csv con la prediccion
pred2016 = model2016.predict(d)

res2016 = c2016['id'].to_frame()
res2016.insert(1,'target', pred2016 , True) 

#res2016.head()

In [ ]:
# 2015

c2015 = fill_nulls(c2015)

# Creo los set de test y train

d = c2015[features]

#Exporto csv con la prediccion
pred2015 = model2015.predict(d)

res2015 = c2015['id'].to_frame()
res2015.insert(1,'target', pred2015 , True) 

#res2015.head()

In [ ]:
# 2014

c2014 = fill_nulls(c2014)

# Creo los set de test y train

d = c2014[features]

#Exporto csv con la prediccion
pred2014 = model2014.predict(d)

res2014 = c2014['id'].to_frame()
res2014.insert(1,'target', pred2014 , True) 

#res2014.head()

In [ ]:
# 2013

c2013 = fill_nulls(c2013)

# Creo los set de test y train

d = c2013[features]

#Exporto csv con la prediccion
pred2013 = model2013.predict(d)

res2013 = c2013['id'].to_frame()
res2013.insert(1,'target', pred2013 , True) 

#res2013.head()

In [ ]:
# 2012

c2012 = fill_nulls(c2012)

# Creo los set de test y train

d = c2012[features]

#Exporto csv con la prediccion
pred2012 = model2012.predict(d)

res2012 = c2012['id'].to_frame()
res2012.insert(1,'target', pred2012 , True) 

#res2012.head()

### Armo el csv final

In [ ]:
resfinal = pd.concat([res2012,res2013,res2014,res2015,res2016], ignore_index=False)
resfinal = resfinal.sort_index()

#Exportamos la prediccion en formato csv sin labels
resfinal.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)
print('CSV creado Exitosamente.')